---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

#### Extract Month, Day, Year in digits

In [2]:
# a = df.str.extract(r'((\d{1,2})[/-]+(\d{,2})[/-](\d{2,4}))') 
a = df.str.extract(r'(?P<Date>(?P<Month>(?:\d{1,2}))[/-]+(?P<Day>(?:\d{1,2}))[/-](?P<Year>(?:\d{4})|\d{2}))')
a

,Date,Month,Day,Year
0,03/25/93,03,25,93
1,6/18/85,6,18,85
2,7/8/71,7,8,71
3,9/27/75,9,27,75
4,2/6/96,2,6,96
5,7/06/79,7,06,79
6,5/18/78,5,18,78
7,10/24/89,10,24,89
8,3/7/86,3,7,86
9,4/10/71,4,10,71


#### Extract Month, Day, Year in words

In [117]:
b = df.str.extract(r'(?P<Date>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:\w|\.)*)[ -](?P<Day>(?:\d{,2}(?:th,|st,|nd,|rd,)?))[,]?[ -](?P<Year>\d{4}))')
# month, day, year
# b = df.str.extract(r'(((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:\w|\.)*)[ -](\d{,2}(?:th,|st,|nd,|rd,)*)[ -](\d{4}))')
b[b.Date.notnull()]
# b.Date.isna().sum()

,Date,Month,Day,Year
194,"April 11, 1990",April,11,1990
195,"May 30, 2001",May,30,2001
196,"Feb 18, 1994",Feb,18,1994
197,"February 18, 1981",February,18,1981
198,"October. 11, 2013",October.,11,2013
199,Jan 24 1986,Jan,24,1986
200,"July 26, 1978",July,26,1978
201,"December 23, 1999",December,23,1999
202,"May 15, 1989",May,15,1989
203,"September 06, 1995",September,06,1995


#### Extract Day, Month, Year in words

In [143]:
c = df.str.extract(r'(?P<Date>(?P<Day>\d{1,2})?[ -]?(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*)[ -,][ ]?(?P<Year>(?:\d{4})))')
#day, month, year
# c = df.str.extract(r'((\d{1,2})?[ -]?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*)[ -,](\d{4}))')
c[c.Date.notnull()]

,Date,Day,Month,Year
125,24 Jan 2001,24,Jan,2001
126,10 Sep 2004,10,Sep,2004
127,26 May 1982,26,May,1982
128,28 June 2002,28,June,2002
129,06 May 1972,06,May,1972
130,25 Oct 1987,25,Oct,1987
131,14 Oct 1996,14,Oct,1996
132,30 Nov 2007,30,Nov,2007
133,28 June 1994,28,June,1994
134,14 Jan 1981,14,Jan,1981


#### Extract only Month/ year

In [144]:
d = df.str.extract(r'(?P<Date>(?P<Month>(?:\d{1,2}))[ /,](?P<Year>(?:\d{4})))')
#month, year
# d = df.str.extract(r'((\d{1,2})[/](\d{4}))')
d = d.where(d.Month.astype(float).isin(range(1, 13)))
d[d.Date.notnull()]

,Date,Month,Year
34,12/2012,12,2012
55,11/1987,11,1987
58,5/2010,5,2010
69,3/1985,3,1985
77,05/1999,05,1999
82,07/1974,07,1974
85,09/1981,09,1981
92,08/2004,08,2004
94,08/1990,08,1990
95,11/1974,11,1974


#### Extract only year

In [148]:
e = df.str.extract(r'(?P<Date>(?P<Year>(?:\d{4})))')
#year
# e = df.str.extract(r'((\d{4}))')
e = e.where(e.astype(float).isin(range(1970,2018)))
e[e.Date.notnull()]

,Date,Year
14,1990,1990
15,2011,2011
17,1976,1976
24,1984,1984
30,1985,1985
34,2012,2012
38,1986,1986
43,2002,2002
45,1998,1998
55,1987,1987


In [149]:
## concatenate all dfs

result = pd.concat([a, b, c, d, e], axis=1, keys=['a','b','c','d','e'])

In [160]:
### helper functionto precess the year into a 4-digit format
def add_prefix(x):
    ans = x
    if len(x) == 2:
        ans = '19'+ans
    return ans

In [161]:
result.loc[:, ('a', 'Year')] = result.loc[:,('a','Year')].str.replace(r'(\d{2,4})', lambda x: add_prefix(x.groups()[0]))

In [152]:
# result.loc[:,('a','Year')] = result.loc[:,('a','Year')] .astype(float)
# result.loc[:,('b','Year')]  = result.loc[:,('b','Year')] .apply(pd.to_numeric, errors= 'ignore')
# result.loc[:,('c','Year')]  = result.loc[:,('c','Year')] .apply(pd.to_numeric, errors= 'ignore')
# result.loc[:,('d','Year')]  = result.loc[:,('d','Year')] .apply(pd.to_numeric, errors= 'ignore')
# result.loc[:,('d','Date')]  = result.loc[:,('d','Date')] .where(result.loc[:,('d','Month')] .isin(range(1, 13)))
# result.loc[:,('e','Date')]  = result.loc[:,('e','Date')] .apply(pd.to_numeric, errors= 'ignore')
# result.loc[:,('e','Date')]  = result.loc[:,('e','Date')] .where(result.loc[:,('e','Date')] .isin(range(1970, 2018)))

In [162]:
### change the year in the date in the d_df if it is different fromt the year extract from e_df
result.loc[:, ('d', 'Date')] = result.loc[:, ('d', 'Date')].where(result.loc[:, ('d','Year')]==result.loc[:,('e','Year')], result.loc[:,('d','Month')]+" "+result.loc[:, ('e','Year')])

### manage the dataframe to consider of the date column in a_df with the order the the following order. (by filling in NAN rows from the other df's)
result.loc[:, ('a', 'Date')] = result.loc[:, ('a', 'Date')].where(~result.loc[:, ('a', 'Date')].isna(), result.loc[:, ('b','Date')])
result.loc[:, ('a', 'Date')] = result.loc[:, ('a', 'Date')].where(~result.loc[:, ('a', 'Date')].isna(), result.loc[:, ('c','Date')])
result.loc[:, ('a', 'Date')] = result.loc[:, ('a', 'Date')].where(~result.loc[:, ('a', 'Date')].isna(), result.loc[:, ('d','Date')])
result.loc[:, ('a', 'Date')] = result.loc[:, ('a', 'Date')].where(~result.loc[:, ('a', 'Date')].isna(), result.loc[:, ('e','Date')])

In [163]:
### check nan counts in a.Date column
result.loc[:,('a','Date')].isna().sum()

0

In [165]:
### Two entries with wrong spelling, and one with unexpected spelling for datetime conversion

result.loc[298, ('a','Date')] = "January 1993"
result.loc[313, ('a','Date')] = "December 1978"
result.loc[321, ('a','Date')] = "2 June, 1999"

In [172]:
### convert to datetime
result.loc[:,('a','Date')] = result.loc[:,('a','Date')].apply(pd.to_datetime, errors = 'ignore')

In [173]:
### use argsort() to easily get the sorted index from datetime
result.loc[:, ('a','Date')].argsort()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: (a, Date), Length: 500, dtype: int64

In [174]:
import numpy as np
def date_sorter(result):
    
    # Your code here  
    return np.argsort(result.loc[:, ('a','Date')])# Your answer here

In [175]:
date_sorter(result)

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: (a, Date), Length: 500, dtype: int64